<div style="line-height:0.5">
<h1 style="color:#FF7C00  ">  Seq2Seq in Tensorflow </h1>
<h4>  </h4>
<h3 style="color:lightblue"> Keywords: </h3>
</div>

In [58]:
import os
import random
import numpy as np
import pandas as pd

In [59]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from sklearn.model_selection import train_test_split

In [56]:
from google.colab import files

In [4]:
uploaded = files.upload()

Saving eng-ita.txt to eng-ita.txt


<h2 style="color:#FF7C00  ">  <u> Example 1 </u> </h2>

In [60]:
english_sentences_simple = [
    "hello", "goodbye", "thank you", "please", "yes", "no", "I love you",
    "how are you", "good morning", "good night", "water", "food", "sun",
    "moon", "star", "book", "computer", "phone", "flower", "tree",
    "house", "car", "bus", "train", "sky", "cloud", "rain", "snow",
    "bird", "cat", "dog", "fish", "mountain", "valley", "ocean", "sea",
    "river", "forest", "desert", "city", "village", "country", "king",
    "queen", "prince", "princess", "happy", "sad", "angry", "excited",
    "bored", "tired", "hungry", "thirsty", "hot", "cold", "big", "small",
    "fast", "slow", "up", "down", "left", "right", "day", "night", "light",
    "dark", "young", "old", "man", "woman", "boy", "girl", "friend", "enemy",
    "song", "dance", "jump", "run", "walk", "stop", "go", "come", "push",
    "pull", "open", "close", "hard", "soft", "short", "tall", "wide", "narrow"
]


In [61]:
italian_sentences_simple = [
    "ciao", "addio", "grazie", "per favore", "sì", "no", "ti amo",
    "come stai", "buongiorno", "buonanotte", "acqua", "cibo", "sole",
    "luna", "stella", "libro", "computer", "telefono", "fiore", "albero",
    "casa", "auto", "autobus", "treno", "cielo", "nuvola", "pioggia", "neve",
    "uccello", "gatto", "cane", "pesce", "montagna", "valle", "oceano", "mare",
    "fiume", "foresta", "deserto", "città", "villaggio", "paese", "re",
    "regina", "principe", "principessa", "felice", "triste", "arrabbiato", "eccitato",
    "annoito", "stanco", "affamato", "assetato", "caldo", "freddo", "grande", "piccolo",
    "veloce", "lento", "su", "giù", "sinistra", "destra", "giorno", "notte", "luce",
    "scuro", "giovane", "vecchio", "uomo", "donna", "ragazzo", "ragazza", "amico", "nemico",
    "canzone", "danza", "salta", "corri", "cammina", "ferma", "vai", "vieni", "spingi",
    "tira", "apri", "chiudi", "duro", "morbido", "corto", "alto", "largo", "stretto"
]


<h3 style="color:#FF7C00  ">  Preprocessing </h3>

In [62]:
tokenizer_eng = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer_eng.fit_on_texts(english_sentences_simple)
english_seq = tokenizer_eng.texts_to_sequences(english_sentences_simple)

tokenizer_ita = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer_ita.fit_on_texts(italian_sentences_simple)
ita_seq = tokenizer_ita.texts_to_sequences(italian_sentences_simple)

max_len_eng = max([len(seq) for seq in english_seq])
max_len_frn = max([len(seq) for seq in ita_seq])

english_seq = keras.preprocessing.sequence.pad_sequences(english_seq, maxlen=max_len_eng, padding='post')
ita_seq = keras.preprocessing.sequence.pad_sequences(ita_seq, maxlen=max_len_frn, padding='post')

<h3 style="color:#FF7C00  ">  Seq2Seq Model </h3>

In [63]:
#### Parameters
embedding_dim = 50
lstm_units = 128
vocab_size_eng = len(tokenizer_eng.word_index) + 1
vocab_size_ita = len(tokenizer_ita.word_index) + 1

In [64]:
###### Encoder
encoder_input = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size_eng, embedding_dim)(encoder_input)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_output, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding)
encoder_states = [encoder_state_h, encoder_state_c]

####### Decoder
decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size_ita, embedding_dim)(decoder_input)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_ita, activation='softmax')
decoder_output = decoder_dense(decoder_output)

model = keras.Model([encoder_input, decoder_input], decoder_output)

In [65]:
# Encoder inference model
encoder_model = keras.Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_inference = Embedding(vocab_size_ita, embedding_dim)(decoder_input)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_inference, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = keras.Model([decoder_input] + decoder_states_inputs, [decoder_outputs] + decoder_states)



In [66]:
""" Training """

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Splitting data for training and testing
eng_train, eng_val, frn_train, frn_val = train_test_split(english_seq, ita_seq, test_size=0.2)

model.fit([eng_train, frn_train[:, :-1]], frn_train[:, 1:],
            validation_data=([eng_val, frn_val[:, :-1]], frn_val[:, 1:]),
            batch_size=2, epochs=100)

Epoch 1/100
38/38 [==============================] - 6s 73ms/step - loss: 2.3371 - accuracy: 0.5040 - val_loss: 1.6101 - val_accuracy: 0.5737
Epoch 2/100
38/38 [==============================] - 0s 11ms/step - loss: 1.7084 - accuracy: 0.5173 - val_loss: 1.4016 - val_accuracy: 0.5789
Epoch 3/100
38/38 [==============================] - 0s 11ms/step - loss: 1.6233 - accuracy: 0.5293 - val_loss: 1.3739 - val_accuracy: 0.5895
Epoch 4/100
38/38 [==============================] - 1s 14ms/step - loss: 1.5630 - accuracy: 0.5387 - val_loss: 1.3266 - val_accuracy: 0.5789
Epoch 5/100
38/38 [==============================] - 0s 7ms/step - loss: 1.5168 - accuracy: 0.5467 - val_loss: 1.3012 - val_accuracy: 0.5947
Epoch 6/100
38/38 [==============================] - 0s 11ms/step - loss: 1.4904 - accuracy: 0.5547 - val_loss: 1.2887 - val_accuracy: 0.5947
Epoch 7/100
38/38 [==============================] - 0s 7ms/step - loss: 1.4670 - accuracy: 0.5613 - val_loss: 1.2501 - val_accuracy: 0.6000
Epoch 8/

In [50]:
def translate(input_sentence):
    # Tokenize and pad the input sentence
    input_seq = tokenizer_eng.texts_to_sequences([input_sentence])
    input_seq = keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_len_eng, padding='post')

    # Get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Start token for the decoder, using the first word in our dictionary as the start point
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 1  # Let's use the first word index as a starting point

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the token with the highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        # Check if the index is in the dictionary
        if sampled_token_index in tokenizer_ita.index_word:
            sampled_char = tokenizer_ita.index_word[sampled_token_index]
            decoded_sentence += ' ' + sampled_char
        else:
            print("does not exist in dictionary!")
            break  # Exit if the index isn't in the dictionary

        # Exit loop if max length is reached
        if len(decoded_sentence.split()) > max_len_frn:
            stop_condition = True

        # Update the target_seq and states for the next loop iteration
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()


In [67]:
input_sentence = "hello"
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 18ms/step
does not exist in dictionary!


'è è è ho tom'

<div style="line-height:0.5">
<h3 style="color:#FF7C00  "> Note: </h3>
</div>
Clearly, the prediction is wrong! Seq2Seq models typically require large datasets to produce accurate translations.

<h2 style="color:#FF7C00  ">  <u> Example 2 </u> </h2>

In [68]:
english_sentences = []
italian_sentences = []

with open("/content/eng-ita.txt", "r", encoding="utf-8") as f:    #./data/eng-ita if not on Colab
    lines = f.readlines()
    for line in lines:
        eng, ita = line.strip().split("\t")
        english_sentences.append(eng)
        italian_sentences.append(ita)


In [69]:
english_sentences[:5], italian_sentences[:5]

(['Hi.', 'Run!', 'Run!', 'Run!', 'Who?'],
 ['Ciao!', 'Corri!', 'Corra!', 'Correte!', 'Chi?'])

In [70]:
%%script echo skipping
""" Use only 20% of the data to avoid memory allocation problems. """

# num_samples = int(0.20 * len(english_sentences))
# # Randomly sample indices
# sampled_indices = random.sample(range(len(english_sentences)), num_samples)

# ## Use indices to sample from lists
# english_sentences_sampled = [english_sentences[i] for i in sampled_indices]
# italian_sentences_sampled = [italian_sentences[i] for i in sampled_indices]

skipping


In [71]:
# Tokenization using TensorFlow's Keras API
tokenizer_eng = tf.keras.preprocessing.text.Tokenizer()
tokenizer_eng.fit_on_texts(english_sentences)
english_sequences = tokenizer_eng.texts_to_sequences(english_sentences)
vocab_size_eng = len(tokenizer_eng.word_index) + 1
max_len_eng = max([len(seq) for seq in english_sequences])

tokenizer_ita = tf.keras.preprocessing.text.Tokenizer()
tokenizer_ita.fit_on_texts(italian_sentences)
italian_sequences = tokenizer_ita.texts_to_sequences(italian_sentences)
vocab_size_ita = len(tokenizer_ita.word_index) + 1
max_len_frn = max([len(seq) for seq in italian_sequences])

# Padding sequences
english_sequences = tf.keras.preprocessing.sequence.pad_sequences(english_sequences, maxlen=max_len_eng, padding='post')
italian_sequences = tf.keras.preprocessing.sequence.pad_sequences(italian_sequences, maxlen=max_len_frn, padding='post')

In [72]:
#### Parameters
embedding_dim = 50
lstm_units = 128
vocab_size_eng = len(tokenizer_eng.word_index) + 1
vocab_size_ita = len(tokenizer_ita.word_index) + 1

In [73]:
###### Encoder
encoder_input = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size_eng, embedding_dim)(encoder_input)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_output, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding)
encoder_states = [encoder_state_h, encoder_state_c]

####### Decoder
decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size_ita, embedding_dim)(decoder_input)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_ita, activation='softmax')
decoder_output = decoder_dense(decoder_output)

model = keras.Model([encoder_input, decoder_input], decoder_output)

In [74]:
# Encoder inference model
encoder_model = keras.Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_inference = Embedding(vocab_size_ita, embedding_dim)(decoder_input)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_inference, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = keras.Model([decoder_input] + decoder_states_inputs, [decoder_outputs] + decoder_states)


In [76]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [142]:
# model.fit([english_sequences, italian_sequences[:,:-1]], keras.utils.to_categorical(italian_sequences[:,1:], num_classes=vocab_size_ita),
#                                                                                     batch_size=64,
#                                                                                     epochs=5,
#                                                                                     validation_split=0.2)


model.fit(
    [english_sequences, italian_sequences[:,:-1]],  # Inputs
    italian_sequences[:,1:],                        # Targets
    batch_size=64,
    epochs=100,
    validation_split=0.2
)

Epoch 1/100
4148/4148 [==============================] - 398s 96ms/step - loss: 0.1578 - accuracy: 0.9656 - val_loss: 0.6621 - val_accuracy: 0.9012
Epoch 2/100
4148/4148 [==============================] - 400s 96ms/step - loss: 0.1320 - accuracy: 0.9698 - val_loss: 0.6496 - val_accuracy: 0.9034
Epoch 3/100
4148/4148 [==============================] - 400s 96ms/step - loss: 0.1125 - accuracy: 0.9732 - val_loss: 0.6496 - val_accuracy: 0.9053
Epoch 4/100
4148/4148 [==============================] - 399s 96ms/step - loss: 0.0975 - accuracy: 0.9758 - val_loss: 0.6530 - val_accuracy: 0.9061
Epoch 5/100
4148/4148 [==============================] - 399s 96ms/step - loss: 0.0860 - accuracy: 0.9779 - val_loss: 0.6543 - val_accuracy: 0.9069
Epoch 6/100
4148/4148 [==============================] - 399s 96ms/step - loss: 0.0769 - accuracy: 0.9796 - val_loss: 0.6597 - val_accuracy: 0.9076
Epoch 7/100
4148/4148 [==============================] - 399s 96ms/step - loss: 0.0697 - accuracy: 0.9809 - val_

In [143]:
def translate(input_sentence):
    # Tokenize and pad the input sentence
    input_seq = tokenizer_eng.texts_to_sequences([input_sentence])
    input_seq = keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_len_eng, padding='post')

    # Get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Start token for the decoder using the most frequent word in Italian sequences
    start_word = tokenizer_ita.index_word[1]
    start_index = tokenizer_ita.word_index[start_word]
    target_seq = np.zeros((1, 1))
    #target_seq[0, 0] = start_index
    target_seq[0, 0] = 1  # Let's use the first word index as a starting point

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the token with the highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        # Check if the index is in the dictionary
        if sampled_token_index in tokenizer_ita.index_word:
            sampled_char = tokenizer_ita.index_word[sampled_token_index]
            decoded_sentence += ' ' + sampled_char
        else:
            print("does not exist in dictionary!")
            break  # Exit if the index isn't in the dictionary

        # Exit loop if max length is reached or if the token is a repeat of the start word
        if (len(decoded_sentence.split()) > max_len_frn) or (sampled_char == start_word):
            stop_condition = True

        # Update the target_seq and states for the next loop iteration
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()


In [144]:
#%%script echo skipping
def translate2(input_sentence):
    # Tokenize and pad the input sentence
    input_seq = tokenizer_eng.texts_to_sequences([input_sentence])
    input_seq = keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_len_eng, padding='post')

    # Get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Initialize the starting sequence for the decoder with a value of 0
    # (this assumes 0 is not a valid token in your dictionary, but just a padding value)
    target_seq = np.zeros((1, 1))

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        print(output_tokens[0, -1, :])

        # Get the token with the highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        # Check if the index is in the dictionary
        if sampled_token_index in tokenizer_ita.index_word:
            sampled_word = tokenizer_ita.index_word[sampled_token_index]
            decoded_sentence += ' ' + sampled_word

            # Exit loop if max length is reached or if we encounter an end token (this can be adjusted)
            if (len(decoded_sentence.split()) > max_len_frn):
                stop_condition = True

        else:
            print("does not exist in dictionary!")
            break

        # Update the target_seq and states for the next loop iteration
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()


skipping


In [145]:
def translate3(input_sentence):
    input_seq = tokenizer_eng.texts_to_sequences([input_sentence])
    input_seq = keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_len_eng, padding='post')

    states_value = encoder_model.predict(input_seq)

    # Change this to use a special <start> token if you added one, else keep as is
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 1

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Print this for debugging
        print(output_tokens)

        predicted_index = np.argmax(output_tokens[0, -1, :])

        if predicted_index in tokenizer_ita.index_word:
            sampled_char = tokenizer_ita.index_word[predicted_index]
            decoded_sentence += ' ' + sampled_char
        else:
            print("Index not in dictionary:", predicted_index)
            break

        if (len(decoded_sentence.split()) > max_len_frn) or (sampled_char == tokenizer_ita.index_word[1]):
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = predicted_index
        states_value = [h, c]

    return decoded_sentence.strip()


In [133]:
input_sentence = "Ask Tom"
predicted_translation = translate3(input_sentence)
predicted_translation

1/1 [==============================] - 0s 21ms/step
[[[7.8913952e-05 9.6949625e-01 7.8210903e-08 ... 7.0891931e-10
   7.8226342e-10 7.0940898e-10]]]


'tom'

In [141]:
input_sentence = "Go slow."
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 18ms/step
[[[6.9520521e-01 7.1514834e-05 7.0451910e-07 ... 2.1985787e-08
   2.1940334e-08 2.1700236e-08]]]
Index not in dictionary: 0


''

In [147]:
input_sentence = "Go slow."
predicted_translation = translate2(input_sentence)
predicted_translation

1/1 [==============================] - 0s 18ms/step
does not exist in dictionary!


'vada'

In [146]:
input_sentence = "Go slow."
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 19ms/step
does not exist in dictionary!


'vada'

In [148]:
input_sentence = "Call me tomorrow."
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 18ms/step
does not exist in dictionary!


'questa domani oggi oggi'

In [150]:
input_sentence = "Call me."
predicted_translation = translate2(input_sentence)
predicted_translation

1/1 [==============================] - 0s 17ms/step
does not exist in dictionary!


'mi'

In [151]:
input_sentence = "I got fat."
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 20ms/step
does not exist in dictionary!


'sono'

In [159]:
input_sentence = "know."
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 19ms/step
does not exist in dictionary!


'lo'

In [153]:
input_sentence = "Get Away!"
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 19ms/step
does not exist in dictionary!


''

In [154]:
input_sentence = "I shaved"
predicted_translation = translate(input_sentence)
predicted_translation

1/1 [==============================] - 0s 19ms/step
does not exist in dictionary!


'la'

In [158]:
input_sentence = "Let's go!"
predicted_translation = translate3(input_sentence)
predicted_translation

1/1 [==============================] - 0s 19ms/step
[[[9.9999070e-01 7.6397136e-18 1.5178664e-18 ... 9.4466676e-13
   8.9033479e-13 9.1798411e-13]]]
Index not in dictionary: 0


''